In [ ]:
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Input, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import applications
from keras.models import Model, Sequential
from keras.regularizers import l2
import gc
import h5py
import numpy as np
import PIL.Image as Image
import os
import pandas
import shutil



/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
def copy(srcPath, dstPath):
    shutil.move(srcPath, dstPath)


current_path = os.getcwd()
vaild_dir = current_path + '/validation'
if (not os.path.exists(vaild_dir)):
    os.mkdir(vaild_dir)

# train_paths = []
# valid_paths = []
csv = pandas.read_csv('driver_imgs_list.csv')
valid_count = 0
groupby = csv.groupby(['subject'])
for key in groupby.indices.keys():
    if (valid_count < 1500):
        pic_arr = groupby.indices[key]
        for index in pic_arr:
            col = csv.iloc[index]
            img_path = col['img']
            clazz = col['classname']
            dstDir = current_path + '/validation/' + clazz
            if (not os.path.exists(dstDir)):
                os.mkdir(dstDir)
            dstPath = dstDir + '/' + img_path
            srcPath = current_path + '/train/' + clazz + '/' + img_path
            copy(srcPath,dstDir)
        print(pic_arr.size)
        valid_count = valid_count + pic_arr.size
    else:
        break

(3017, 224, 224, 3)
load finish!!!


In [ ]:
mean = [103.939, 116.779, 123.68]

def normalize(x):
    x = x.astype('float32')
    x[:, :, 0] = (x[:, :, 0] - mean[0])
    x[:, :, 1] = (x[:, :, 1] - mean[1])
    x[:, :, 2] = (x[:, :, 2] - mean[2])
    return x

train_generator = ImageDataGenerator(
    preprocessing_function=normalize)

test_generator = ImageDataGenerator(preprocessing_function=normalize)


def get_custom_model(num_class):
    base_model = applications.VGG16(include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[:4]:
        print(layer.name)
        layer.trainable = False
    x = GlobalAveragePooling2D(name='global')(base_model.get_layer('block5_pool').output)
    #     x = Dense(20, activation='linear', name='fc1', kernel_regularizer=l2(0.01))(x)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dense(num_class, activation='softmax', name='pred')(x)
    model = Model(input=base_model.input, output=x)
    sgd = SGD(lr=1e-1, momentum=0.9, decay=1e-6, nesterov=True)
    #     sgd = SGD(lr=1e-10)
    model.compile(optimizer=sgd,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    return model


custom_model = get_custom_model(10)

print("compile finish!!!")
current_path = os.getcwd()

custom_model.fit_generator(
    train_generator.flow_from_directory(class_mode='categorical', batch_size=128,
                                        directory=current_path + '/train',
                                        target_size=(224, 224)),
    use_multiprocessing=True,
    epochs=20,
    shuffle=True,
    validation_data=test_generator.flow_from_directory(class_mode='categorical', batch_size=128,
                                                       target_size=(224, 224),
                                                       directory=current_path + '/validation'),
    verbose=1)

custom_model.save_weights('weight_1.meta')


input_3
block1_conv1
block1_conv2
block1_pool
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


Epoch 1/20
152/152 [==============================] - 497s 3s/step - loss: 2.3565 - acc: 0.1061 - val_loss: 2.3019 - val_acc: 0.1236
Epoch 2/20
152/152 [==============================] - 475s 3s/step - loss: 2.3013 - acc: 0.1090 - val_loss: 2.3014 - val_acc: 0.1236
Epoch 3/20
152/152 [==============================] - 475s 3s/step - loss: 2.3008 - acc: 0.1091 - val_loss: 2.3012 - val_acc: 0.1236
Epoch 4/20
152/152 [==============================] - 476s 3s/step - loss: 2.3004 - acc: 0.1090 - val_loss: 2.3010 - val_acc: 0.1236
Epoch 5/20
 18/152 [==>...........................] - ETA: 6:32 - loss: 2.2983 - acc: 0.1155